In [1]:
import geopandas as gpd
from sqlalchemy import create_engine
from MyCreds.mycreds import Capstone_AWS_PG       #from local site-packages
from geoalchemy2 import Geometry
import shapely
from bs4 import BeautifulSoup
import numpy as np
import fiona

engine = create_engine(f'postgresql+psycopg2://{Capstone_AWS_PG.username}:{Capstone_AWS_PG.password}@{Capstone_AWS_PG.host}/capstone', echo=False)

sql_alc_cnxn = engine.connect()

In [2]:
gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'

In [3]:
food_market_df = gpd.read_file('data/nea-market-and-food-centre/nea-market-and-food-centre-kml.kml')

In [4]:
# get stall details

def process_geodf(df, tag, list_of_col_names):
    add_cols = {key:[] for key in list_of_col_names}
    for i in df.iterrows():
        soup = BeautifulSoup(i[1][1]).findAll(tag)
        add_cols[list_of_col_names[0]].append(soup[0].text)
        add_cols[list_of_col_names[1]].append(soup[1].text)
        add_cols[list_of_col_names[2]].append(soup[2].text)
        add_cols[list_of_col_names[3]].append(soup[3].text)
        add_cols[list_of_col_names[4]].append(soup[6].text)
    return df.assign(**add_cols)

In [5]:
food_market_df = process_geodf(food_market_df,'td',['total_stalls','market_stalls','cooked_food_stalls','postal','location'])
food_market_df

,Name,Description,geometry,total_stalls,market_stalls,cooked_food_stalls,postal,location
0,kml_1,<center><table><tr><th colspan='2' align='cent...,POINT Z (103.85296 1.33602 0.00000),36,0,36,310075,"Blk 75, Lorong 5 Toa Payoh, S(310075)"
1,kml_2,<center><table><tr><th colspan='2' align='cent...,POINT Z (103.81833 1.28778 0.00000),122,122,0,150079,"Blk 79, Redhill Lane, S(150079)"
2,kml_3,<center><table><tr><th colspan='2' align='cent...,POINT Z (103.80762 1.27336 0.00000),40,0,40,100079,"Blk 79, Telok Blangah Drive, S(100079)"
3,kml_4,<center><table><tr><th colspan='2' align='cent...,POINT Z (103.88710 1.32783 0.00000),59,43,16,370080,"Blk 80, Circuit Road, S(370080)"
4,kml_5,<center><table><tr><th colspan='2' align='cent...,POINT Z (103.80790 1.27389 0.00000),64,64,0,100082,"Blk 82, Telok Blangah Drive, S(100082)"
...,...,...,...,...,...,...,...,...
105,kml_106,<center><table><tr><th colspan='2' align='cent...,POINT Z (103.87001 1.32146 0.00000),211,127,84,330069,"Blk 69, Geylang Bahru, S(330069)"
106,kml_107,<center><table><tr><th colspan='2' align='cent...,POINT Z (103.80564 1.31625 0.00000),108,68,40,260007,"Blk 7, Empress Road, S(260007)"
107,kml_108,<center><table><tr><th colspan='2' align='cent...,POINT Z (103.84651 1.37217 0.00000),123,78,45,560724,"Blk 724, Ang Mo Kio Ave 6, S(560724)"
108,kml_109,<center><table><tr><th colspan='2' align='cent...,POINT Z (103.76425 1.30381 0.00000),195,135,60,120726,"Blk 726, Clementi West Street 2, S(120726)"


In [6]:
# # we also swap around lat longs - leave Z
# food_market_df['geometry']=food_market_df.geometry.map(lambda point: shapely.ops.transform(lambda x, y, z: (y, x, z), point))
# food_market_df

In [7]:
food_market_df.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [8]:
food_market_df.to_postgis('food_and_markets',engine, if_exists='replace', index=False, dtype={'geometry': Geometry(geometry_type='POINT Z', srid= 4326)})